### The way the model in trianed in the file SpamHamClassificationProjectTFIDF.ipynb caused the data leakage
#### To overcome the problem of Data leakage problem, once the data cleaning is done, immediately go for trian test split, this will keep the test data complete unknown to the model.

## Spam ham Classification Project using TFIDF

In [1]:
import pandas as pd
messages = pd.read_csv("..\\smsSpamCollection\\SMSSpamCollection.txt", sep='\t', names=['label','message'])

In [2]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Now we need to clean the dataset or preprocess the data
### Steps include:
* removing numbers from text
* tokenize
* stemming
* remove stopwords
* find word vector

In [3]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spurusho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:
word_tokenize(messages['message'][0])

['Go',
 'until',
 'jurong',
 'point',
 ',',
 'crazy',
 '..',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 '...',
 'Cine',
 'there',
 'got',
 'amore',
 'wat',
 '...']

In [6]:
corpus = []
for i in range(len(messages)):
    message = re.sub('[^a-zA-Z]',' ', messages['message'][i])
    message = message.lower()
    message = word_tokenize(message)
    message = [lemmatizer.lemmatize(word, pos='v') for word in message if word not in stopwords.words('english')]
    message = ' '.join(message)
    corpus.append(message)

In [7]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine get amore wat',
 'ok lar joke wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf live around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune callers press copy friends callertune',
 'winner value network customer select receivea prize reward claim call claim code kl valid hours',
 'mobile months u r entitle update latest colour mobiles camera free call mobile update co free',
 'gon na home soon want talk stuff anymore tonight k cry enough today',
 'six chance win cash pound txt csh send cost p day days tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(messages['label'])

In [9]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=31)

In [10]:
# Create the bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
# for binary bag of words enable binary true
tf = TfidfVectorizer(max_features=2500, ngram_range=(1,2))

In [11]:
X_train = tf.fit_transform(X_train).toarray()
X_test = tf.transform(X_test).toarray()

In [12]:
tf.vocabulary_

{'call': 238,
 'do': 544,
 'min': 1298,
 'later': 1111,
 'secret': 1815,
 'admirer': 23,
 'look': 1189,
 'make': 1238,
 'contact': 411,
 'find': 682,
 'reveal': 1757,
 'think': 2105,
 'ur': 2239,
 'special': 1954,
 'secret admirer': 1816,
 'admirer look': 24,
 'look make': 1192,
 'make contact': 1240,
 'contact find': 414,
 'find reveal': 684,
 'reveal think': 1758,
 'think ur': 2110,
 'ur special': 2257,
 'special call': 1956,
 'plz': 1605,
 'note': 1462,
 'anyone': 70,
 'mobile': 1331,
 'co': 357,
 'amp': 59,
 'ask': 99,
 'type': 2218,
 'lt': 1218,
 'gt': 863,
 'coz': 440,
 'iz': 1045,
 'attempt': 109,
 'use': 2272,
 'sim': 1885,
 'card': 290,
 'nokia': 1448,
 'motorola': 1365,
 'verify': 2286,
 'call mobile': 255,
 'lt gt': 1220,
 'gt lt': 868,
 'well': 2375,
 'class': 348,
 'pm': 1606,
 'hope': 956,
 'day': 487,
 'ok': 1485,
 'coffee': 364,
 'good': 838,
 'stay': 1981,
 'late': 1110,
 'tomorrow': 2155,
 'time': 2126,
 'place': 1586,
 'always': 56,
 'hope day': 957,
 'like': 1160,
 

In [13]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [14]:
y_pred = spam_detect_model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test, y_pred)

0.9730941704035875

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       962
           1       0.98      0.82      0.89       153

    accuracy                           0.97      1115
   macro avg       0.97      0.91      0.94      1115
weighted avg       0.97      0.97      0.97      1115



## here i observed that overall accuracy was reduced by 1 percent by preventing the data leak. That really matters. GOOD!!!

### Thats it for this, we just cleaned the data and applied the naivebayes algorithm for classfication